In [8]:
import pandas as pd
from database import Database
from datetime import datetime
from Filter.Market import Market


In [9]:
db = Database()
q_data=db.get_finance_report_q()
m_data=db.get_finance_report_m()
s_data = db.get_stock('2023/09/01','2023/09/16')
mk_data = db.get_market_index('2023/09/01','2023/09/16')
company_data =db.get_company_basic()
ch_data = db.get_chips('2023/09/01','2023/09/16')

c:\Users\User\Desktop\中央資管\論文\自己的論文\Data_Browsing\database.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_from_db[columns_to_fill] = filtered_df_from_db[columns_to_fill].fillna(0)


In [10]:
selected_companies=[1101,4971,9940]

In [11]:
def getFliterStock(s_data,selected_companies):

        #找出符合條件的公司股價資料
        stock_fliter_data = s_data[(s_data['證券代碼'].isin(selected_companies))]

        # 先將 DataFrame 根據 "證券代碼" 和 "日期" 進行排序
        stock_fliter_data = stock_fliter_data.sort_values(by=["證券代碼", "日期"], ascending=[True, True])

        #取得有資料的首日
        first_date = stock_fliter_data.head(1)['日期']

        #取得區間內最高價
        max_prices= stock_fliter_data.groupby('證券代碼')['最高價(元)'].max()
        max_prices = max_prices.rename('區間最高價(元)').reset_index()

        start_prices = stock_fliter_data[stock_fliter_data['日期'] == first_date.iloc[0]]
        result_df = pd.merge(max_prices, start_prices, on='證券代碼', how='left')
        # 計算 "區間最高股價變化"
        result_df['區間最大價差'] = (result_df['區間最高價(元)'] - result_df['最低價(元)'])

        # 計算 "區間最高股價變化率"
        result_df['區間最大價差%'] = ((result_df['區間最高價(元)'] - result_df['最低價(元)'])/result_df['最低價(元)']*100).round(2)
        return result_df  

In [13]:
#取得首日價量資訊以及計算 區間最大價差 和 區間最大價差%
stock_data = getFliterStock(s_data,selected_companies)
#取得其他table資料
# m_data = self.m_data
# q_data = self.q_data
# chips_data= self.chips_data
# market_data = self.market_data

# 先將相關數據確定轉為日期格式
m_data['月報所屬日期'] = pd.to_datetime(m_data['月報所屬日期'])
m_data['營收發布日'] = pd.to_datetime(m_data['營收發布日'])
q_data['季報所屬日期'] = pd.to_datetime(q_data['季報所屬日期'])
q_data['季報發布日'] = pd.to_datetime(q_data['季報發布日'])
stock_data['日期'] = pd.to_datetime(stock_data['日期'])


# m_data 按照 '營收發布日' 以及 q_data 按照 '季報發布日' 從新到舊排序
m_data = m_data.sort_values(by='營收發布日', ascending=False)
q_data = q_data.sort_values(by='季報發布日', ascending=False)

# 查找 '發布日' 對應的 '所屬日期'
def find_next_date(row, data, date_column, target_date_column):
    company_id = int(row['證券代碼'])
    date = row['日期']
    
    # 從 data 中篩選出相同 '證券代碼' 的紀錄
    correspond_company_data = data[data['證券代碼'] == company_id]
    
    for _, m_row in correspond_company_data.iterrows():
        if date > m_row[date_column]:
            return m_row[target_date_column]
    
    return None

# 將 '月報所屬日期' 列填充為對應的 '營收所屬日期'
stock_data['月報所屬日期'] = stock_data.apply(find_next_date, args=(m_data, '營收發布日', '月報所屬日期',), axis=1)

# 將 '季報所屬日期' 列填充為對應的 '季報所屬日期'
stock_data['季報所屬日期'] = stock_data.apply(find_next_date, args=(q_data, '季報發布日', '季報所屬日期',), axis=1)
final_merged_df = pd.merge(stock_data, m_data, on=['月報所屬日期', '證券代碼'], how='left').merge(q_data, on=['季報所屬日期', '證券代碼'], how='left').merge(ch_data, on=['日期', '證券代碼'], how='left').merge(company_data, on=['證券代碼'], how='left').merge(mk_data,left_on=['日期', '上市別'], right_on=['日期', '指數名稱'], how='left')
final_merged_df.to_excel('final_merged_df.xlsx')
final_merged_df

,證券代碼,區間最高價(元),日期,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),5日均價(元),10日均價(元),...,指數收盤價(元),指數成交量(千股),指數5日均價(元),指數10日均價(元),指數20日均價(元),指數60日均價(元),指數5日均量,指數10日均量,指數20日均量,指數60日均量
0,1101,35.50,2023-09-01,35.1,35.45,35.0,35.25,19428.0,35.22,35.26,...,16644.94,5858177.0,16626.44,16578.06,16597.69,16906.65,5731910.0,5879095.0,6236167.0,7031436.0
1,4971,63.80,2023-09-01,58.6,59.00,58.0,58.40,91.0,57.60,57.12,...,213.68,1632809.0,211.07,209.61,210.24,218.33,1383854.0,1352950.0,1480626.0,1765150.0
2,9940,28.95,2023-09-01,28.6,28.75,28.6,28.75,82.0,28.66,28.61,...,16644.94,5858177.0,16626.44,16578.06,16597.69,16906.65,5731910.0,5879095.0,6236167.0,7031436.0
